In [9]:
import glob
from pathlib import Path
import pandas as pd

from model_training.measures import accuracy, f1_score

In [2]:
MODELS_PATH = '../models'

## Compare the results for different techniques

In [ ]:
results = {}

for predictions_file in glob.glob(f'{MODELS_PATH}/**/predictions.csv', recursive=True):
    df = pd.read_csv(predictions_file)
    y_true = df["y_true"].values
    y_pred = df["y_pred"].values

    accuracy_score = accuracy(y_true, y_pred)
    f1_score_value = f1_score(y_true, y_pred)
    results[Path(predictions_file).parent.stem] = {
        'accuracy': accuracy_score,
        'f1_score': f1_score_value
    }

results_df = pd.DataFrame(results).T
results_df

,accuracy,f1_score
resnet18_tsrc_spectrograms,0.378577,0.372848
simple_cnn_tsrc_spectrograms,0.803680,0.808253
simple_cnn_tsrc_spectrograms_cleaned,0.823534,0.825102
simple_cnn_tsrc_spectrograms_cleaned_sharpened,0.824599,0.826996
